# Event Detection

To achieve real-time event detection in Python, we have utilised the pyaudio library to capture live audio from the microphone and analyse the audio data to identify events or patterns. The detect_events function monitors the audio data and triggers an alert when the audio exceeds a predefined threshold. This enables the user to be promptly notified of detected events while the audio stream is being monitored.

This code has been tested and is ready to deploy to a live testing environment. 

Author: cpavia

In [10]:
########################################################################################
# library imports
########################################################################################

# disable warnings
import warnings
warnings.filterwarnings("ignore")

# environment settings
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

# processing related libraries
import librosa

# real-time audio event detection
import pyaudio
import wave


In [11]:
def detect_event(audio_samples, threshold):
    # Perform event detection logic here
    # Function to detect an event using spectral centroid
    spectral_centroids = librosa.feature.spectral_centroid(y=audio_samples, sr=44100)[0]
    avg_spectral_centroid = np.mean(spectral_centroids)
    return avg_spectral_centroid < threshold

def record_and_save_audio(output_folder, recording_duration=10, threshold=0.5):
    # Parameters for audio recording
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 44100

    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    frames = []
    event_detected = False
    
    print("Listening for events...")
    for i in range(0, int(RATE / CHUNK * recording_duration)):
        audio_data = stream.read(CHUNK)
        frames.append(audio_data)
        
        # Convert the audio data to integers
        audio_samples = np.frombuffer(audio_data, dtype=np.int16)

        # Convert the integer samples to float format (-1.0 to 1.0 range)
        audio_samples = audio_samples / np.iinfo(np.int16).max
        
        # Detect event
        if not event_detected and detect_event(audio_samples, threshold):
            event_detected = True
            print("Event detected!")

    stream.stop_stream()
    stream.close()
    p.terminate()
    
    # Save the recorded audio if an event was detected
    if event_detected:
        os.makedirs(output_folder, exist_ok=True)
        output_file = os.path.join(output_folder, "event_recording.wav")
        wf = wave.open(output_file, "wb")
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b"".join(frames))
        wf.close()
        print("Event recording saved to:", output_file)
    else:
        print("No event found")

if __name__ == "__main__":
    
    # Set the output folder where the event recording will be saved
    output_folder = r'C:\path\to\the\directory'

    # Set the duration (in seconds) for recording
    recording_duration = 5

    # Set the threshold for event detection (adjust based on your audio input)
    threshold = 3500

    record_and_save_audio(output_folder, recording_duration, threshold)

Listening for events...
Event detected!
Event recording saved to: C:\path\to\the\directory\event_recording.wav
